In [4]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, r2_score

import mlflow

import pandas as pd
import seaborn as sns
import numpy as np
import os
import matplotlib.pyplot as plt
import re

In [5]:
train = pd.read_csv('application_train.csv')
test = pd.read_csv('application_test.csv')

###LABEL ENCODING

le = LabelEncoder()

for col in train:
    if train[col].dtype == 'object':
# If 2 or fewer unique categories
        if len(list(train[col].unique())) <= 2:
            le.fit(train[col])
            train[col] = le.transform(train[col])
            test[col] = le.transform(test[col])

###ONE HOT ENCODING

train = pd.get_dummies(train)
test = pd.get_dummies(test)

#train = train.dropna(axis='columns')
#test = test.dropna(axis='columns')

labels = train['TARGET']

###ALIGN TRAIN AND TEST

train, test = train.align(test, join='inner', axis=1)
train['TARGET'] = labels

###DEFINE X AND Y

X = train.drop(columns=['TARGET'])
y = train['TARGET']

In [6]:
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 91.93%


In [90]:
#The numbers in the DAYS_BIRTH column are negative because they are recorded relative to the current loan application. To see these stats in years, we can mutliple by -1 and divide by the number of days in a year:
(train['DAYS_BIRTH'] / -365).describe()

count    307511.000000
mean         43.936973
std          11.956133
min          20.517808
25%          34.008219
50%          43.150685
75%          53.923288
max          69.120548
Name: DAYS_BIRTH, dtype: float64

In [82]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     93342
           1       0.46      0.04      0.07      8137

    accuracy                           0.92    101479
   macro avg       0.69      0.52      0.51    101479
weighted avg       0.88      0.92      0.89    101479



In [95]:
confusion_matrix(y_test,predictions)

array([[92996,   346],
       [ 7845,   292]], dtype=int64)

In [65]:
###RANDOM FOREST

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

classifier = RandomForestClassifier(n_estimators=20, random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [7]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.07998699238266045
Mean Squared Error: 0.07998699238266045
Root Mean Squared Error: 0.282819717103777


In [8]:
print(accuracy_score(y_test, y_pred))

0.9200130076173395


In [9]:
print(confusion_matrix(y_test,y_pred))

[[93362     0]
 [ 8117     0]]


In [ ]:
a = pd.Series(y_pred)
a.value_counts()

In [ ]:
labels.value_counts()

In [8]:
metrics.r2_score(y_test,predictions)

-0.09438891829722573

In [17]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(2)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [18]:
missing_values_table(train)

Your selected dataframe has 122 columns.
There are 67 columns that have missing values.


Missing Values  % of Total Values
COMMONAREA_MEDI                   214865              69.87
COMMONAREA_AVG                    214865              69.87
COMMONAREA_MODE                   214865              69.87
NONLIVINGAPARTMENTS_MEDI          213514              69.43
NONLIVINGAPARTMENTS_MODE          213514              69.43
...                                  ...                ...
EXT_SOURCE_2                         660               0.21
AMT_GOODS_PRICE                      278               0.09
AMT_ANNUITY                           12               0.00
CNT_FAM_MEMBERS                        2               0.00
DAYS_LAST_PHONE_CHANGE                 1               0.00

[67 rows x 2 columns]

In [66]:
missing_values = missing_values_table(train)
missing_values.head()

Your selected dataframe has 202 columns.
There are 23 columns that have missing values.


Missing Values  % of Total Values
FLOORSMAX_AVG                         153020              49.76
FLOORSMAX_MODE                        153020              49.76
FLOORSMAX_MEDI                        153020              49.76
YEARS_BEGINEXPLUATATION_AVG           150007              48.78
YEARS_BEGINEXPLUATATION_MODE          150007              48.78

In [ ]:
train.shape

In [ ]:
test.shape

In [57]:
###FEATURE SELECTION

train_missing = (train.isnull().sum() / len(train)).sort_values(ascending = False)
train_missing.head(20)

COMMONAREA_MEDI             0.698723
COMMONAREA_AVG              0.698723
COMMONAREA_MODE             0.698723
NONLIVINGAPARTMENTS_AVG     0.694330
NONLIVINGAPARTMENTS_MODE    0.694330
NONLIVINGAPARTMENTS_MEDI    0.694330
LIVINGAPARTMENTS_MEDI       0.683550
LIVINGAPARTMENTS_AVG        0.683550
LIVINGAPARTMENTS_MODE       0.683550
FLOORSMIN_MEDI              0.678486
FLOORSMIN_MODE              0.678486
FLOORSMIN_AVG               0.678486
YEARS_BUILD_MODE            0.664978
YEARS_BUILD_MEDI            0.664978
YEARS_BUILD_AVG             0.664978
OWN_CAR_AGE                 0.659908
LANDAREA_AVG                0.593767
LANDAREA_MODE               0.593767
LANDAREA_MEDI               0.593767
BASEMENTAREA_AVG            0.585160
dtype: float64

In [58]:
test_missing = (test.isnull().sum() / len(test)).sort_values(ascending = False)
test_missing.head(20)

COMMONAREA_MODE             0.687161
COMMONAREA_MEDI             0.687161
COMMONAREA_AVG              0.687161
NONLIVINGAPARTMENTS_MEDI    0.684125
NONLIVINGAPARTMENTS_AVG     0.684125
NONLIVINGAPARTMENTS_MODE    0.684125
LIVINGAPARTMENTS_MODE       0.672493
LIVINGAPARTMENTS_AVG        0.672493
LIVINGAPARTMENTS_MEDI       0.672493
FLOORSMIN_AVG               0.666051
FLOORSMIN_MEDI              0.666051
FLOORSMIN_MODE              0.666051
OWN_CAR_AGE                 0.662892
YEARS_BUILD_MODE            0.652757
YEARS_BUILD_AVG             0.652757
YEARS_BUILD_MEDI            0.652757
LANDAREA_MODE               0.579641
LANDAREA_MEDI               0.579641
LANDAREA_AVG                0.579641
BASEMENTAREA_MEDI           0.567065
dtype: float64

In [59]:
train_missing = (train.isnull().sum() / len(train)).sort_values(ascending = False)
train_missing.head(20)

test_missing = (test.isnull().sum() / len(test)).sort_values(ascending = False)
test_missing.head(20)

train_missing = train_missing.index[train_missing > 0.5]
test_missing = test_missing.index[test_missing > 0.5]

all_missing = list(set(set(train_missing) | set(test_missing)))
print('There are %d columns with more than 50%% missing values' % len(all_missing))

There are 38 columns with more than 50% missing values


In [63]:
train.shape

(307511, 202)

In [62]:
for i in train_missing:
    train = train.drop([i], axis = 'columns')
    test = test.drop([i], axis = 'columns')

In [64]:
test.shape

(48744, 201)